In [1]:
import numpy as np
import gensim
import re

In [2]:
clean = ['warandpeace.txt', 'annakarenina.txt', 'idiot.txt', 'onegin.txt', 'jivago.txt', 'ktulhu_demo.txt',
        'atthemountainsofmadness.txt', 'brotherskaramasovi.txt', 'journeytomoscow.txt', 'oblomov.txt']
medium = ['choke.txt', 'fightclub.txt', 'lullaby.txt', 'masterandmargaret.txt', 'messenger.txt',
         'pulpfiction.txt', 'survivor.txt', 'oneflewover.txt']
slang = ['mitiki.txt', 'rastamans.txt', 'chapaev_demo.txt']

In [3]:
names = [clean, medium, slang]
texts = []
fulls = []
sets = []

for group in names:
    docs = []
    full = ''
    for book in group:
        corpus = []
        with open('books/' + book) as file:
            for line in file:
                corpus.append(line)

        corpus = [el for el in corpus if el != '\n']
        corpus = [re.sub(r'[^\w\s]','',el)[:-1] for el in corpus]
        for sentence in corpus:
            full += ' ' + sentence
        docs += [el.lower().split() for el in corpus]
        
    words = []
    for sentence in docs:
        words += sentence
    sets.append(set(words))
    texts += docs
    fulls.append(full)

In [4]:
model = gensim.models.Word2Vec(texts, size=100, iter=70, workers=4, min_count=1)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer()
tfidf = vec.fit_transform(fulls)
npidf = tfidf.toarray()
fnames = vec.get_feature_names()

In [6]:
def logic(string):
    for word in string.split():
        try:
            index = fnames.index(word)
            idf = npidf[:, index]
            if (idf[1] > 100 * idf[0]) or (idf[2] > 100 * idf[0]):
                best = '0'
                best_res = -100
                for gword in sets[0]:
                    score = model.wv.similarity(word, gword)
                    if score > best_res:
                        best_res = score
                        best = gword
                print(word, ' ---> ', best)
        except:
            pass